In [12]:
import os
import random
import h5py
import json
import torch
import logging
from torch.utils.data import Dataset
import numpy as np
# from .build import DATASETS
# from utils.logger import print

def rotate_point_cloud_z(pc):
    """ Randomly rotate the point clouds to augment the dataset """
    rotation_angle = np.random.uniform() * 2 * np.pi
    cosval = np.cos(rotation_angle)
    sinval = np.sin(rotation_angle)
    rotation_matrix = np.array([[cosval, -sinval, 0],
                                [sinval, cosval, 0],
                                [0, 0, 1]])
    rotated_data = np.dot(pc, rotation_matrix)
    return rotated_data

def jitter_point_cloud(pc, sigma=0.01, clip=0.05):
    """ Randomly jitter points. jittering is per point. """
    N, C = pc.shape
    assert(clip > 0)
    jittered_data = np.clip(sigma * np.random.randn(N, C), -1 * clip, clip)
    jittered_data += pc
    return jittered_data

def random_scale_point_cloud(pc, scale_low=0.8, scale_high=1.25):
    """ Randomly scale the point cloud. Scale is per shape. """
    scale = np.random.uniform(scale_low, scale_high)
    return pc * scale


# @DATASETS.register_module()
class ShapeNetPartH5(Dataset):
    """
    Dataloader for the HDF5 version of ShapeNetPart.
    This is the standard dataset format used in PointNet/PointNet++ and subsequent works.
    """
    # 50-class mapping for ShapeNetPart
    seg_classes = {
        'Airplane': [0, 1, 2, 3], 'Bag': [4, 5], 'Cap': [6, 7], 'Car': [8, 9, 10, 11],
        'Chair': [12, 13, 14, 15], 'Earphone': [16, 17, 18], 'Guitar': [19, 20, 21],
        'Knife': [22, 23], 'Lamp': [24, 25, 26, 27], 'Laptop': [28, 29],
        'Motorbike': [30, 31, 32, 33, 34, 35], 'Mug': [36, 37], 'Pistol': [38, 39, 40],
        'Rocket': [41, 42, 43], 'Skateboard': [44, 45, 46], 'Table': [47, 48, 49]
    }
    
    # Mapping from category name to the class index (0-15)
    classes_map = {
        'Airplane': 0, 'Bag': 1, 'Cap': 2, 'Car': 3, 'Chair': 4, 'Earphone': 5,
        'Guitar': 6, 'Knife': 7, 'Lamp': 8, 'Laptop': 9, 'Motorbike': 10,
        'Mug': 11, 'Pistol': 12, 'Rocket': 13, 'Skateboard': 14, 'Table': 15
    }

    def __init__(self, config):
        self.root = config.DATA_PATH
        self.npoints = config.N_POINTS
        self.split = config.subset
        self.use_augmentation = (self.split == 'train')

        self.all_points = []
        self.all_seg_labels = []
        self.all_cls_labels = []

        # Find all H5 files for the given split (train/test/val)
        h5_files = [f for f in os.listdir(self.root) if f.endswith('.h5') and self.split in f]
        if not h5_files:
            raise FileNotFoundError(f"No H5 files found for split '{self.split}' in '{self.root}'")
        
        print(f"Loading H5 files for '{self.split}' split: {h5_files}")

        for h5_filename in sorted(h5_files):
            f = h5py.File(os.path.join(self.root, h5_filename), 'r')
            points = f['data'][:]
            seg_labels = f['seg'][:] 
            cls_labels = f['label'][:]
            f.close()
            
            self.all_points.append(points)
            self.all_seg_labels.append(seg_labels)
            self.all_cls_labels.append(cls_labels)

        # Concatenate data from all loaded files
        self.all_points = np.concatenate(self.all_points, axis=0)
        self.all_seg_labels = np.concatenate(self.all_seg_labels, axis=0)
        self.all_cls_labels = np.concatenate(self.all_cls_labels, axis=0).squeeze() # Squeeze to make it 1D

        print(f'The size of {self.split} data is {len(self.all_points)}')
        print(f'Number of points per sample: {self.npoints}')
        
        self.classes = self.classes_map
        
    def __len__(self):
        return len(self.all_points)

    def __getitem__(self, index):
        points = self.all_points[index][:self.npoints].copy()
        seg_labels = self.all_seg_labels[index][:self.npoints].copy()
        cls_label = self.all_cls_labels[index].copy()

        # Augmentation is applied only to the training set
        if self.use_augmentation:
            points = rotate_point_cloud_z(points)
            points = jitter_point_cloud(points)
            points = random_scale_point_cloud(points)
        
        # Normalize points
        points = self.pc_normalize(points)


        return (
            torch.from_numpy(points).float(),
            torch.from_numpy(np.array([cls_label])).long(), # Wrap in array for consistent shape
            torch.from_numpy(seg_labels).long()
        )

    @staticmethod
    def pc_normalize(pc):
        centroid = np.mean(pc, axis=0)
        pc = pc - centroid
        m = np.max(np.sqrt(np.sum(pc**2, axis=1)))
        pc = pc / (m + 1e-9)
        return pc
    


if __name__ == "__main__":
    import sys
    import random
    # project_root = os.path.dirname(os.path.dirname(os.path.abspath(__file__)))
    # sys.path.insert(0, project_root)

    class DummyConfig1:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'train'  # can be 'train' or 'test'

    config = DummyConfig1()
    
    print(f"--- Testing ShapeNetPart HDF5 Dataset ---")
    print(f"Loading data from: {config.DATA_PATH}")
    print(f"Subset: {config.subset}, Points per sample: {config.N_POINTS}")
    
    # 2. --- Instantiate the dataset ---
    try:
        dataset_train = ShapeNetPartH5(config)
        
    except Exception as e:
        print(f"\n[ERROR] Failed to initialize dataset. Please check the DATA_PATH in the script.")
        print(f"Details: {e}")
        exit()
    # 3. --- Get and inspect a random sample ---
    if len(dataset_train) == 0:
        print("\n[ERROR] The dataset is empty. No data was found. Check the DATA_PATH and dataset structure.")
    else:
        print(f"\nDataset loaded successfully with {len(dataset_train)} samples.")
        
        # The new dataloader has a map from name to class index. We need the reverse for printing.
        idx_to_name_map = {v: k for k, v in dataset_train.classes_map.items()}
        
        # Get a random item
        random_index = random.randint(0, len(dataset_train) - 1)
        print(f"Fetching random sample at index: {random_index}")
        
        # The __getitem__ method returns a tuple of tensors
        points_tensor, cls_label_tensor, seg_labels_tensor = dataset_train[random_index]   # first is all points and their coordinates second is object category third is label pr points
        
        # --- NEW LOGIC TO GET CATEGORY NAME (continued) ---
        # Get the category index from the tensor, then look up its name
        cat_idx = cls_label_tensor.item()
        cat_name = idx_to_name_map.get(cat_idx, f"UnknownCategory_{cat_idx}")
        
        print(f"\n--- Sample Details for Category: {cat_name} ---")
        
        # Check shapes
        print(f"Points tensor shape:      {points_tensor.shape} (Expected: [{config.N_POINTS}, 3])")
        print(f"Class label tensor shape:   {cls_label_tensor.shape} (Expected: [1])")
        print(f"Seg labels tensor shape:  {seg_labels_tensor.shape} (Expected: [{config.N_POINTS}])")
        
        # Check dtypes
        print(f"\nPoints tensor dtype:      {points_tensor.dtype}")
        print(f"Class label tensor dtype:   {cls_label_tensor.dtype}")
        print(f"Seg labels tensor dtype:  {seg_labels_tensor.dtype}")
        
        # Check content
        class_label = cls_label_tensor.item()
        print(f"\nClass label value: {class_label}")
        
        # This part of the check remains the same and is still the most important one
        unique_labels, counts = np.unique(seg_labels_tensor.numpy(), return_counts=True)
        print("\n--- Segmentation Label Analysis ---")
        print("This is the most crucial check. If you see multiple labels, the loader is working.")
        print(f"Unique part labels found in sample: {unique_labels}")
        print(f"Point counts for each label:      {counts}")

        if len(unique_labels) <= 1:
            print("\n[WARNING] Only one unique label was found. The part segmentation data may not be loading correctly.")
        else:
            print("\n[SUCCESS] Multiple unique labels found. The dataset appears to be loading part data correctly.")


        print(type(dataset_train))

--- Testing ShapeNetPart HDF5 Dataset ---
Loading data from: /kaggle/input/shapenetpart/shapenetpart_hdf5_2048
Subset: train, Points per sample: 2048
Loading H5 files for 'train' split: ['train3.h5', 'train5.h5', 'train1.h5', 'train4.h5', 'train2.h5', 'train0.h5']
The size of train data is 12137
Number of points per sample: 2048

Dataset loaded successfully with 12137 samples.
Fetching random sample at index: 4787

--- Sample Details for Category: Table ---
Points tensor shape:      torch.Size([2048, 3]) (Expected: [2048, 3])
Class label tensor shape:   torch.Size([1]) (Expected: [1])
Seg labels tensor shape:  torch.Size([2048]) (Expected: [2048])

Points tensor dtype:      torch.float32
Class label tensor dtype:   torch.int64
Seg labels tensor dtype:  torch.int64

Class label value: 15

--- Segmentation Label Analysis ---
This is the most crucial check. If you see multiple labels, the loader is working.
Unique part labels found in sample: [47 48 49]
Point counts for each label:      [

In [13]:
class DummyConfig2:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'test' 



config2 = DummyConfig2()
dataset_test = ShapeNetPartH5(config2)
print(len(dataset_test))

print(dataset_train[0])

Loading H5 files for 'test' split: ['test1.h5', 'test0.h5']
The size of test data is 2874
Number of points per sample: 2048
2874
(tensor([[-0.6945, -0.2199,  0.0348],
        [ 0.9427,  0.3208, -0.0264],
        [-0.1488, -0.0528, -0.7667],
        ...,
        [-0.0698, -0.0109,  0.5672],
        [ 0.2032, -0.0029, -0.0208],
        [-0.7838, -0.2563,  0.0051]]), tensor([0]), tensor([0, 0, 1,  ..., 1, 0, 0]))


In [14]:
category_to_parts = dataset_train.seg_classes
category_to_parts
category_to_index = dataset_train.classes_map
category_to_index
index_to_category = {v:k for k,v in category_to_index.items()}
index_to_category

{0: 'Airplane',
 1: 'Bag',
 2: 'Cap',
 3: 'Car',
 4: 'Chair',
 5: 'Earphone',
 6: 'Guitar',
 7: 'Knife',
 8: 'Lamp',
 9: 'Laptop',
 10: 'Motorbike',
 11: 'Mug',
 12: 'Pistol',
 13: 'Rocket',
 14: 'Skateboard',
 15: 'Table'}

In [15]:
class DummyConfig3:
        def __init__(self):
            # --- IMPORTANT ---
            # This path must point to the folder containing the .h5 files
            self.DATA_PATH = '/kaggle/input/shapenetpart/shapenetpart_hdf5_2048'
            self.N_POINTS = 2048
            self.subset = 'val' 

config3 = DummyConfig3()
dataset_val = ShapeNetPartH5(config3)
print(len(dataset_val))

Loading H5 files for 'val' split: ['val0.h5']
The size of val data is 1870
Number of points per sample: 2048
1870


In [18]:
import plotly.express as px
import pandas as pd
import numpy as np

def visualize_point_cloud_interactive(sample):
    """
    sample: a tuple from your dataset, e.g., (points, labels) or (points, labels, seg)
    """
    points,classes,labels = sample
    # labels = dataset_train[1] if len(sample) > 1 else np.zeros(points.shape[0])
    
    # Convert to NumPy if torch tensor
    # if not isinstance(points, np.ndarray):
    #     points = points.numpy()
    # if not isinstance(labels, np.ndarray):
    #     labels = labels.numpy()
    
    df = pd.DataFrame({
        "x": points[:, 0],
        "y": points[:, 1],
        "z": points[:, 2],
        "label": labels
    })
    
    fig = px.scatter_3d(
        df, x="x", y="y", z="z", color="label",
        labels={"label": "Classes"}, opacity=0.7
    )
    fig.update_traces(marker=dict(size=3, line=dict(width=1, color='DarkSlateGrey')), selector=dict(mode='markers'))
    fig.update_layout(
        title="Interactive Point Cloud Visualization",
        scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'),
        legend_title="Labels"
    )
    fig.show()

# Example usage
visualize_point_cloud_interactive(dataset_train[1000])
# visualize_point_cloud_interactive(dataset_train[300])


In [22]:
"""
Minimal Point Transformer-like network implementation in PyTorch.
This is an educational, end-to-end implementation showing:
 - Point Transformer layer (local attention with relative positional encoding)
 - Transition Down (farthest point sampling + grouping + aggregation)
 - Transition Up (interpolation + skip connections)
 - A simple encoder-decoder segmentation network

Notes:
 - This is a readable reference implementation, not heavily optimized.
 - You can replace random sampling with more advanced sampling or accelerate kNN with faiss / torch-cluster.
 - For real training use, consider batching, data augmentation and GPU-optimized neighbor search.

Author: ChatGPT (example code)
"""

import torch
import torch.nn as nn
import torch.nn.functional as F

# ----------------------------- helpers -----------------------------

def pairwise_distance(x):
    # x: (B, C, N)
    xx = torch.sum(x**2, dim=1, keepdim=True)  # (B,1,N)
    inner = -2 * torch.matmul(x.transpose(2,1), x)  # (B,N,N)
    dist = xx + inner + xx.transpose(2,1)
    return dist


def knn(x, k):
    # x: (B, C, N) -> return indices (B, N, k)
    # uses pairwise distance on feature or coords
    with torch.no_grad():
        dist = pairwise_distance(x)  # (B,N,N)
        _, idx = dist.topk(k=k, dim=-1, largest=False)  # (B,N,k)
    return idx


def index_points(points, idx):
    # points: (B, C, N) or (B, N, C)
    # idx: (B, S, k) or (B, S)
    # returns gathered points
    device = points.device
    B = points.shape[0]

    if points.dim() == 3:  # (B, C, N)
        _, C, N = points.shape
        idx_flat = idx.reshape(B, -1)  # (B, S*k)
        idx_flat = idx_flat.unsqueeze(1).expand(-1, C, -1)  # (B,C,S*k)
        gathered = torch.gather(points, 2, idx_flat)  # (B,C,S*k)
        out = gathered.reshape(B, C, *idx.shape[1:])  # (B,C,S,k) or (B,C,S)
        return out
    else:
        raise ValueError("index_points expects points as (B,C,N)")


def farthest_point_sample(xyz, npoint):
    # xyz: (B, 3, N)
    # returns indices (B, npoint)
    device = xyz.device
    B, _, N = xyz.shape
    centroids = torch.zeros(B, npoint, dtype=torch.long, device=device)
    distance = torch.ones(B, N, device=device) * 1e10
    farthest = torch.randint(0, N, (B,), dtype=torch.long, device=device)
    batch_indices = torch.arange(B, dtype=torch.long, device=device)
    for i in range(npoint):
        centroids[:, i] = farthest
        centroid_xyz = xyz[batch_indices, :, farthest].unsqueeze(-1)  # (B,3,1)
        dist = torch.sum((xyz - centroid_xyz) ** 2, dim=1)  # (B,N)
        mask = dist < distance
        distance[mask] = dist[mask]
        farthest = torch.max(distance, dim=1)[1]
    return centroids


def three_nn_interp(xyz_src, xyz_dst, feats_src):
    # xyz_src: (B,3,NS), xyz_dst: (B,3,ND), feats_src: (B,C,NS)
    # returns feats_dst: (B,C,ND) by inverse-distance weighted interpolation of 3 nearest
    B, _, NS = xyz_src.shape
    _, _, ND = xyz_dst.shape
    # compute pairwise dist (B,ND,NS)
    xyz_src_t = xyz_src.transpose(1,2)  # (B,NS,3)
    xyz_dst_t = xyz_dst.transpose(1,2)  # (B,ND,3)
    dist = torch.cdist(xyz_dst_t, xyz_src_t)  # (B,ND,NS)
    dists, idx = dist.topk(3, largest=False)  # (B,ND,3)
    dists = torch.clamp(dists, min=1e-10)
    weight = 1.0 / dists
    weight = weight / torch.sum(weight, dim=-1, keepdim=True)  # (B,ND,3)
    # gather features
    idx = idx.long()
    idx_flat = idx.reshape(B, -1)  # (B, ND*3)
    # gather feats_src (B,C,NS) -> (B,C,ND,3)
    gathered = index_points(feats_src, idx.view(B, ND, 3))  # (B,C,ND,3)
    weighted = gathered * weight.unsqueeze(1)  # (B,C,ND,3)
    out = torch.sum(weighted, dim=-1)  # (B,C,ND)
    return out

# ------------------------- Point Transformer Layer -------------------------

class PosEncodingMLP(nn.Module):
    def __init__(self, in_dim=3, out_dim=64):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, out_dim),
            nn.ReLU(inplace=True),
            nn.Linear(out_dim, out_dim)
        )
    def forward(self, x):
        return self.net(x)


class PointTransformerLayer(nn.Module):
    def __init__(self, d_model, k=16):
        super().__init__()
        self.k = k
        self.to_q = nn.Linear(d_model, d_model)
        self.to_k = nn.Linear(d_model, d_model)
        self.to_v = nn.Linear(d_model, d_model)
        self.pos_mlp = PosEncodingMLP(3, d_model)
        self.attn_mlp = nn.Sequential(
            nn.Linear(d_model, d_model),
            nn.ReLU(inplace=True),
            nn.Linear(d_model, d_model)
        )
        self.gamma = nn.Parameter(torch.zeros(1))
        self.norm = nn.LayerNorm(d_model)

    def forward(self, xyz, features):
        # xyz: (B,3,N), features: (B,C,N)
        B, C, N = features.shape
        assert C == self.to_q.in_features

        # compute neighbors on coordinates (or on features) -> indices (B,N,k)
        idx = knn(xyz, self.k)  # uses pairwise on xyz

        # gather neighbor xyz and features
        neighbor_xyz = index_points(xyz, idx)  # (B,3,N,k)
        center_xyz = xyz.unsqueeze(-1)  # (B,3,N,1)
        relative_pos = (neighbor_xyz - center_xyz).permute(0,2,3,1)  # (B,N,k,3)

        neighbor_feats = index_points(features, idx)  # (B,C,N,k)
        center_feats = features.unsqueeze(-1)  # (B,C,N,1)

        # projections
        q = self.to_q(features.permute(0,2,1))  # (B,N,C)
        k = self.to_k(neighbor_feats.permute(0,2,3,1))  # (B,N,k,C)
        v = self.to_v(neighbor_feats.permute(0,2,3,1))  # (B,N,k,C)

        # positional encoding
        pos_enc = self.pos_mlp(relative_pos)  # (B,N,k,C)

        # compute attention logits
        # q - k + pos
        q_expand = q.unsqueeze(2)  # (B,N,1,C)
        attn_input = q_expand - k + pos_enc
        attn = self.attn_mlp(attn_input)  # (B,N,k,C)
        attn = F.softmax(attn, dim=2)  # softmax over neighbors

        # value + pos
        v = v + pos_enc

        # weighted sum
        out = torch.sum(attn * v, dim=2)  # (B,N,C)
        out = out.permute(0,2,1)  # (B,C,N)

        # residual + norm
        out = self.gamma * out + features
        out = out.permute(0,2,1)
        out = self.norm(out)
        out = out.permute(0,2,1)
        return out

# ------------------------- Transition Down / Up -------------------------

class TransitionDown(nn.Module):
    def __init__(self, in_channels, out_channels, npoint, k=16):
        super().__init__()
        self.npoint = npoint
        self.k = k
        self.mlp = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, 1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, xyz, feats):
        # xyz: (B,3,N), feats: (B,C,N)
        idx = farthest_point_sample(xyz, self.npoint)  # (B, npoint)
        # gather sampled coords
        sampled_xyz = index_points(xyz, idx)  # (B,3,npoint)
        # find k neighbors of sampled points from original set
        # for simplicity: compute knn on coords of original -> get neighbors per original; we want neighbors per sampled
        # compute pairwise distances between sampled and all points
        sampled_xyz_t = sampled_xyz.transpose(1,2)  # (B,npoint,3)
        xyz_t = xyz.transpose(1,2)  # (B,N,3)
        dist = torch.cdist(sampled_xyz_t, xyz_t)  # (B,npoint,N)
        _, neighbor_idx = dist.topk(k=self.k, dim=-1, largest=False)  # (B,npoint,k)
        # gather neighbor features
        neighbor_feats = index_points(feats, neighbor_idx)  # (B,C,npoint,k)
        # aggregate (max pooling)
        aggregated, _ = torch.max(neighbor_feats, dim=-1)  # (B,C,npoint)
        out_feats = self.mlp(aggregated)  # (B, out_channels, npoint)
        return sampled_xyz, out_feats


class TransitionUp(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super().__init__()
        self.mlp = nn.Sequential(
            nn.Conv1d(in_channels + skip_channels, out_channels, 1),
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, xyz_src, feats_src, xyz_dst, feats_skip):
        # xyz_src: (B,3,NS) coarse
        # feats_src: (B,C_src,NS)
        # xyz_dst: (B,3,ND) dense
        # feats_skip: (B,C_skip,ND)
        # interpolate feats_src -> ND
        feats_interp = three_nn_interp(xyz_src, xyz_dst, feats_src)  # (B, C_src, ND)
        cat = torch.cat([feats_interp, feats_skip], dim=1)  # (B, C_src + C_skip, ND)
        out = self.mlp(cat)
        return out

# ------------------------- Full Network (Segmentation) -------------------------

class PointTransformerSeg(nn.Module):
    def __init__(self, num_classes=13, k=16):
        super().__init__()
        self.k = k
        # input embed
        self.input_mlp = nn.Sequential(
            nn.Conv1d(3, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True)
        )

        # encoder
        self.pt1 = PointTransformerLayer(64, k=k)
        self.td1 = TransitionDown(64, 128, npoint=1024, k=k)

        self.pt2 = PointTransformerLayer(128, k=k)
        self.td2 = TransitionDown(128, 256, npoint=256, k=k)

        self.pt3 = PointTransformerLayer(256, k=k)
        self.td3 = TransitionDown(256, 512, npoint=64, k=k)

        self.bottleneck = PointTransformerLayer(512, k=k)

        # decoder
        self.tu1 = TransitionUp(512, 256, 256)
        self.ptd1 = PointTransformerLayer(256, k=k)

        self.tu2 = TransitionUp(256, 128, 128)
        self.ptd2 = PointTransformerLayer(128, k=k)

        self.tu3 = TransitionUp(128, 64, 64)
        self.ptd3 = PointTransformerLayer(64, k=k)

        # head
        self.head = nn.Sequential(
            nn.Conv1d(64, 64, 1),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Conv1d(64, num_classes, 1)
        )

    def forward(self, xyz):
        # xyz: (B,3,N)
        B, _, N = xyz.shape
        # xyz = xyz.permute(0, 2, 1)
        feats = self.input_mlp(xyz)  # (B,64,N)

        # enc1
        feats = self.pt1(xyz, feats)
        xyz1, feats1 = self.td1(xyz, feats)  # (B,3,1024),(B,128,1024)

        # enc2
        feats1 = self.pt2(xyz1, feats1)
        xyz2, feats2 = self.td2(xyz1, feats1)  # (B,3,256),(B,256,256)

        # enc3
        feats2 = self.pt3(xyz2, feats2)
        xyz3, feats3 = self.td3(xyz2, feats2)  # (B,3,64),(B,512,64)

        # bottleneck
        feats3 = self.bottleneck(xyz3, feats3)

        # dec1
        feats_up1 = self.tu1(xyz3, feats3, xyz2, feats2)
        feats_up1 = self.ptd1(xyz2, feats_up1)

        # dec2
        feats_up2 = self.tu2(xyz2, feats_up1, xyz1, feats1)
        feats_up2 = self.ptd2(xyz1, feats_up2)

        # dec3
        feats_up3 = self.tu3(xyz1, feats_up2, xyz, feats)
        feats_up3 = self.ptd3(xyz, feats_up3)

        out = self.head(feats_up3)  # (B, num_classes, N)
        return out


# ------------------------- quick smoke test -------------------------
if __name__ == '__main__':
    B = 2
    N = 1024
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = PointTransformerSeg(num_classes=13, k=16).to(device)
    xyz = torch.rand(B, 3,N, device=device)
    out = model(xyz)
    print('output shape', out.shape)  # (B, num_classes, N)


output shape torch.Size([2, 13, 1024])


In [23]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from tqdm import tqdm

# Hyperparameters
NUM_CLASSES = 50  # total segmentation classes
BATCH_SIZE = 16
EPOCHS = 1
LR = 0.001
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

# Data loaders
train_loader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(dataset_val, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Model
# from model_point_transformer import PointTransformerSeg  # your implementation
model = PointTransformerSeg(num_classes=NUM_CLASSES).to(DEVICE)

optimizer = torch.optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

# Training loop
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct, total = 0, 0

    for points, category, seg in tqdm(train_loader):
        points = points.to(DEVICE, dtype=torch.float32).permute(0, 2, 1)  # (B, 3, N)
        seg = seg.to(DEVICE, dtype=torch.long)                             # (B, N)
        category = category.to(DEVICE, dtype=torch.long)                   # (B,)
        
        optimizer.zero_grad()
        pred = model(points)  # (B, num_classes, N)
        pred = pred.permute(0, 2, 1).contiguous()  # (B, N, num_classes)
    
        loss = F.cross_entropy(pred.view(-1, NUM_CLASSES), seg.view(-1))
        loss.backward()
        optimizer.step()


        total_loss += loss.item()
        pred_labels = pred.argmax(-1)
        correct += (pred_labels == seg).sum().item()
        total += seg.numel()

    scheduler.step()
    acc = correct / total
    print(f"Epoch {epoch+1}/{EPOCHS}, Loss: {total_loss/len(train_loader):.4f}, Acc: {acc:.4f}")

    # Optional: evaluation
    model.eval()
    with torch.no_grad():
        test_acc = 0
        for points, category,seg in val_loader:
            points = points.to(DEVICE, dtype=torch.float32).permute(0, 2, 1)
            seg = seg.to(DEVICE)
            pred = model(points).permute(0, 2, 1)
            test_acc += (pred.argmax(-1) == seg).float().mean().item()
        print(f"Test Acc: {test_acc / len(val_loader):.4f}")


100%|██████████| 759/759 [12:10<00:00,  1.04it/s]

Epoch 1/1, Loss: 1.5109, Acc: 0.5801


Test Acc: 0.7779


In [34]:
import torch
import torch.nn.functional as F
import time
import numpy as np
import psutil
import os
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
def compute_metrics(model, dataloader, num_classes, device):
    model.eval()
    total_correct, total_seen = 0, 0
    total_loss = 0.0
    all_iou_per_instance = []
    part_intersection = np.zeros(num_classes)
    part_union = np.zeros(num_classes)
   

    start_time = time.time()
    with torch.no_grad():
        for points, category, seg in dataloader:
            points = points.to(DEVICE, dtype=torch.float32).permute(0, 2, 1)
            category = category.to(device, dtype=torch.long).squeeze()
            seg = seg.to(device, dtype=torch.long)

            pred = model(points)  # (B,num_classes,N)
            
            pred = pred.permute(0, 2, 1).contiguous()  # (B, N, num_classes)
    
            loss = F.cross_entropy(pred.view(-1, NUM_CLASSES), seg.view(-1))
            
            
            pred_choice = pred.argmax(dim=2)  # (B, N)
            pred_flat = pred_choice.view(-1)  # (B*N,)
            seg_flat = seg.view(-1)           # (B*N,)
            
            correct = (pred_flat == seg_flat).sum().item()
            
            
            total_loss += loss.item()
            total_correct +=correct
            
            
            total_seen += seg_flat.numel()

            # --- Compute IoU per instance ---
            preds_np = pred_choice.cpu().numpy().reshape(points.size(0), -1)
            seg_np = seg.cpu().numpy().reshape(points.size(0), -1)
            for shape_idx in range(points.size(0)):
                part_iou = []
                for part in np.unique(seg_np[shape_idx]):
                    I = np.sum((preds_np[shape_idx] == part) & (seg_np[shape_idx] == part))
                    U = np.sum((preds_np[shape_idx] == part) | (seg_np[shape_idx] == part))
                    if U == 0:
                        iou = 1.0
                    else:
                        iou = I / float(U)
                    part_iou.append(iou)
                    part_intersection[part] += I
                    part_union[part] += U
                all_iou_per_instance.append(np.mean(part_iou))

    # --- Metrics ---
    total_time = time.time() - start_time
    num_samples = len(dataloader.dataset)
    avg_inference_time = total_time / num_samples

    overall_acc = total_correct / total_seen
    instance_miou = np.mean(all_iou_per_instance)
    class_miou = np.mean(part_intersection / np.maximum(part_union, 1e-6))

    # --- Memory ---
    process = psutil.Process(os.getpid())
    memory_mb = process.memory_info().rss / 1024 ** 2  # in MB

    # --- Model size ---
    param_size = 0
    for param in model.parameters():
        param_size += param.nelement() * param.element_size()
    buffer_size = 0
    for buffer in model.buffers():
        buffer_size += buffer.nelement() * buffer.element_size()
    model_size_mb = (param_size + buffer_size) / 1024 ** 2

    return {
        'loss': total_loss / len(dataloader),
        'overall_acc': overall_acc,
        'instance_miou': instance_miou,
        'class_miou': class_miou,
        'avg_inference_time': avg_inference_time,
        'memory_mb': memory_mb,
        'model_size_mb': model_size_mb
    }


In [35]:
metrics = compute_metrics(model, test_loader, NUM_CLASSES, DEVICE)

print(f"\nValidation Metrics:")
print(f"Loss:              {metrics['loss']:.4f}")
print(f"Overall Accuracy:  {metrics['overall_acc']:.4f}")
print(f"Instance mIoU:     {metrics['instance_miou']:.4f}")
print(f"Class mIoU:        {metrics['class_miou']:.4f}")
print(f"Inference Time:    {metrics['avg_inference_time']*1000:.2f} ms per sample")
print(f"Memory Usage:      {metrics['memory_mb']:.2f} MB")
print(f"Model Size:        {metrics['model_size_mb']:.2f} MB\n")



Validation Metrics:
Loss:              1.0028
Overall Accuracy:  0.7054
Instance mIoU:     0.5470
Class mIoU:        0.2732
Inference Time:    40.21 ms per sample
Memory Usage:      2104.12 MB
Model Size:        11.70 MB



## comparison brooo

In [37]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_single_segmentation_with_category(points, seg_gt, seg_pred, category_name="Unknown", shape_idx=0):
    """
    points: (N, 3) numpy array of point cloud coordinates
    seg_gt: (N,) numpy array of ground truth labels
    seg_pred: (N,) numpy array of predicted labels
    category_name: string label for the object category
    shape_idx: integer index of the object
    """

    fig = make_subplots(
        rows=1, cols=2,
        specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
        subplot_titles=[
            f"Ground Truth - {category_name} (Object {shape_idx})",
            f"Prediction - {category_name} (Object {shape_idx})"
        ]
    )

    # Ground truth segmentation
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=seg_gt,
            colorscale='Viridis',
            opacity=0.8,
            colorbar=dict(title="GT Labels")
        ),
        name="Ground Truth"
    ), row=1, col=1)

    # Predicted segmentation
    fig.add_trace(go.Scatter3d(
        x=points[:, 0],
        y=points[:, 1],
        z=points[:, 2],
        mode='markers',
        marker=dict(
            size=2,
            color=seg_pred,
            colorscale='Rainbow',
            opacity=0.8,
            colorbar=dict(title="Predicted Labels")
        ),
        name="Prediction"
    ), row=1, col=2)

    fig.update_layout(
        title=f"Segmentation Comparison for Category: {category_name}",
        width=1200,
        height=600
    )

    fig.show()


In [40]:
model.eval()
device = DEVICE
num_classes = NUM_CLASSES

# Keep track of which categories we've already visualized
shown_categories = set()
test_loader = DataLoader(dataset_test, batch_size=BATCH_SIZE, shuffle=False)
with torch.no_grad():
    for points, category, seg in test_loader:
    # move to GPU
        points = points.to(DEVICE, dtype=torch.float32).permute(0, 2, 1)  # (B, 3, N)
        category = category.to(DEVICE, dtype=torch.long).squeeze()
        seg = seg.to(DEVICE, dtype=torch.long)  # (B, N)
    
        # forward
        pred = model(points)  # (B, num_classes, N)
        pred = pred.permute(0, 2, 1).contiguous()  # (B, N, num_classes)
    
        # get predicted class per point
        pred_choice = pred.argmax(dim=2)  # (B, N)
    
        # flatten for metrics (optional)
        pred_flat = pred_choice.view(-1)  # (B*N,)
        seg_flat = seg.view(-1)           # (B*N,)
    
        # convert to numpy for IoU computation etc.
        preds_np = pred_choice.cpu().numpy()  # (B, N)
        seg_np = seg.cpu().numpy()

        # Iterate over batch items
        for b in range(points.size(0)):
            cat_id = category[b].item()
            cat_name = index_to_category[cat_id]

            # Skip if already visualized this category
            if cat_name in shown_categories:
                continue

            # Plot one sample of this category
            plot_single_segmentation_with_category(
                points[b].cpu().numpy(),
                seg_np[b],
                preds_np[b],
                category_name=cat_name,
                shape_idx=b
            )

            shown_categories.add(cat_name)
            print(f"✅ Shown category: {cat_name} ({len(shown_categories)}/16)")

            # Stop once all 16 have been shown
            if len(shown_categories) == 16:
                print("\n🎉 Displayed one sample from each of the 16 categories. Done!")
                break

        if len(shown_categories) == 16:
            break

        


✅ Shown category: Car (1/16)


✅ Shown category: Table (2/16)


✅ Shown category: Lamp (3/16)


✅ Shown category: Guitar (4/16)


✅ Shown category: Chair (5/16)


...

✅ Shown category: Bag (12/16)


✅ Shown category: Earphone (13/16)


✅ Shown category: Skateboard (14/16)


✅ Shown category: Cap (15/16)


✅ Shown category: Rocket (16/16)

🎉 Displayed one sample from each of the 16 categories. Done!
